# Introduction to Python 
# Homework #6
- This homework will be graded
- You MUST submit on CourseWorks2 before it closes
    - Verify your submission succeeded
    - Email submissions are not accepted 
    

# Academic Honesty
* The computer science department has strict polices. Check
the department [web page](http://www.cs.columbia.edu/education/honesty) for details. 
- Do not look at anybody else's source code. Do not show anybody
your source, or leave your source where somebody could see it.
You MUST write your own code.
- For this class, feel free to discuss issues with other people, but suggest waiting an hour or two after a discussion, before writing your code.
-  Cases of non original source will be refered to the Judical Committee.



# Problem 1&2 - Python Help from a zipfile
- Extract data directly from a zipfile without unziping it
- download file 'python-3.7.2-docs-text.zip' from Courseworks, and place it in the same directory as this notebook
    - do NOT unzip the zip file.
- use [ZipFile module](https://docs.python.org/3.7/library/zipfile.html) to access the zip file. 
    
only need to use these methods
    - ZipFile(path)
        - 'open' the zip file and read from it
        - like file 'open', should use 'with' to manage it
    - infolist()
        - returns list of the ZipInfo objects in zip archive
    - is_dir() 
        - does the ZipInfo represent a directory?
    - open()
        - opens a file in the zip for reading
        - use 'with'
- example below shows everything you need to know about ZipFile


In [3]:
import zipfile

# zip file should be in the same directory as this notebook
zippath = 'python-3.7.2-docs-text.zip'

# check first 10 entries in zip
# note nested 'with' - very nice
with zipfile.ZipFile(zippath) as zf:
    for j, info in zip(range(10), zf.infolist()):
        if info.is_dir():
            # ignore directories
            print('skip dir :', info.filename)
            continue
        with zf.open(info) as fd:
            lines = fd.readlines()
        print('file name:', info.filename, 'file line count:', len(lines))

skip dir : python-3.7.2-docs-text/
file name: python-3.7.2-docs-text/contents.txt file line count: 8205
skip dir : python-3.7.2-docs-text/howto/
file name: python-3.7.2-docs-text/howto/cporting.txt file line count: 382
file name: python-3.7.2-docs-text/howto/unicode.txt file line count: 751
file name: python-3.7.2-docs-text/howto/index.txt file line count: 42
file name: python-3.7.2-docs-text/howto/argparse.txt file line count: 727
file name: python-3.7.2-docs-text/howto/instrumentation.txt file line count: 404
file name: python-3.7.2-docs-text/howto/urllib2.txt file line count: 609
file name: python-3.7.2-docs-text/howto/sorting.txt file line count: 301


# define Help class
- write class that extracts info from the python help files, packaged in a zip file
- ```__init__(self, zippath)``` method should read in the contents of the zip file located at zippath, and save the data in the help object
    - convert input to unicode
    - strip off the 'python-3.7.2-docs-text/' at the begining of the file name
- three user methods - they all should 'print' their results, and return 'None'. this will make it easy to use them in a script later
- if a file line contains the substring arg one or more times, that is a 'hit'
    - counts(self, substring) method - checks all lines of all files for hits. print the total number of hits
    - files(self, substring) method - for each file with hits, print file name and # of hits
    - grep(self, substring) method - for each line with a hit, print file name, line number, and line

- note that like the linux 'grep' command, multiple appearances of the search string in a single line is scored as ONE hit - we are counting the lines that have a hit(s). 
    - if line = 'dog cat dog' , and the search string is 'dog', that is ONE hit


In [132]:
class Help:
    def __init__(self, zippath):
        self.path = zippath[:]
        self.data = {}
        
        # store list of files found in the stream - store in unicode not binary
        # strip out the root dir - also don't save dirs
        with zipfile.ZipFile(self.path) as zstream:
            for file in zstream.infolist():
                if file.is_dir(): continue
                
                # not a dir so now append to self.data but strip ziproot
                fname = file.filename.split('/')
                fname = '/'.join(fname[1:])
                
                self.data[fname] = []
                
                with zstream.open(file) as fdata:
                    for line in fdata.readlines():
                        self.data[fname].append(line.decode('utf-8'))
    
    def counts(self, substring):
        count = 0
        for fname, data in self.data.items():
            for line in data:
                if substring in line:
                    count += 1
        print(count)
        return None
    
    def files(self, substring):
        fnames = {}
        
        for fname, data in self.data.items():
            count = 0
            for line in data:
                if substring in line:
                    count += 1
                    fnames.update({fname: count})
        
        for fname, count in fnames.items():
            print(f'{fname} : {count}')
        return None
    
    def grep(self, substring):
        for fname, data in self.data.items():
            linenum = -1
            for line in data:
                linenum += 1
                if substring in line:
                    print(f'{fname}({linenum}): {line.strip()}')
        return None
                    
        

In [137]:
h = Help(zippath)

In [138]:
h.counts('rpartition')

13


In [139]:
h.files('rpartition')

whatsnew/2.7.txt : 1
whatsnew/changelog.txt : 2
whatsnew/2.5.txt : 4
whatsnew/3.2.txt : 2
library/importlib.txt : 1
library/stdtypes.txt : 3


In [140]:
h.grep('rpartition')

whatsnew/2.7.txt(916): * The "split()", "replace()", "rindex()", "rpartition()", and
whatsnew/changelog.txt(3334): * bpo-20047: Bytearray methods partition() and rpartition() now
whatsnew/changelog.txt(7251): * bpo-20047: Bytearray methods partition() and rpartition() now
whatsnew/2.5.txt(975): "rpartition(sep)" methods that simplify a common use case.
whatsnew/2.5.txt(984): "rpartition(sep)" also returns a 3-tuple but starts searching from
whatsnew/2.5.txt(995): >>> 'www.python.org'.rpartition('.')
whatsnew/2.5.txt(997): >>> 'www.python.org'.rpartition(':')
whatsnew/3.2.txt(2379): extension = name.rpartition('.')[2]
whatsnew/3.2.txt(2419): also used by "rfind()", "rindex()", "rsplit()" and "rpartition()".
library/importlib.txt(1673): parent_name, _, child_name = absolute_name.rpartition('.')
library/stdtypes.txt(1549): str.rpartition(sep)
library/stdtypes.txt(2319): bytes.rpartition(sep)
library/stdtypes.txt(2320): bytearray.rpartition(sep)


In [141]:
h.counts('linux')

44


In [142]:
h.files('linux')

install/index.txt : 3
whatsnew/changelog.txt : 4
whatsnew/3.5.txt : 4
whatsnew/2.0.txt : 3
whatsnew/3.3.txt : 5
whatsnew/2.6.txt : 1
whatsnew/2.3.txt : 2
whatsnew/2.5.txt : 1
whatsnew/3.2.txt : 1
library/ssl.txt : 1
library/sysconfig.txt : 2
library/test.txt : 1
library/select.txt : 1
library/errno.txt : 1
library/sys.txt : 5
library/platform.txt : 2
library/ossaudiodev.txt : 1
library/socket.txt : 1
tutorial/interpreter.txt : 1
distutils/apiref.txt : 2
extending/building.txt : 2


In [143]:
h.grep('linux')

install/index.txt(673): [GCC 2.96 20000731 (Red Hat Linux 7.3 2.96-112)] on linux2
install/index.txt(677): ['', '/usr/local/lib/python2.3', '/usr/local/lib/python2.3/plat-linux2',
install/index.txt(725): '/www/python/lib/pythonX.Y/plat-linux2', ...]".
whatsnew/changelog.txt(1812): * bpo-28167: The function "platform.linux_distribution" and
whatsnew/changelog.txt(11467): messages of platform.dist() and platform.linux_distribution(). Patch
whatsnew/changelog.txt(14035): messages of platform.dist() and platform.linux_distribution(). Patch
whatsnew/changelog.txt(15739): * bpo-1322: platform.dist() and platform.linux_distribution()
whatsnew/3.5.txt(2049): * "<os>" is always "linux-gnu", except for extensions built to
whatsnew/3.5.txt(2051): "linux- gnu32" (and "<architecture>" will be "x86_64").
whatsnew/3.5.txt(2129): The "platform.dist()" and "platform.linux_distribution()" functions
whatsnew/3.5.txt(2294): on linux 3.6 and greater.
whatsnew/2.0.txt(412): >>> 'hostname'.replace('os', 'lin

In [144]:
for w in ['rpartition', 'linux','universal', 
          'mode', 'zipfile', 'zipfiles']:
    h.counts(w)

13
44
67
1307
128
9


# define python script 'click-zip' based on Help class
- use your Help class from above verbatim
    - build the zippath into the script
- use click to parse command line args, and call the appropriate method
    - '-c' should run counts method
    - '-h' shows help
    - '-g' should run grep method
    - '-f' should run file method
- sample useage below

# click-example script

In [ ]:
#!/usr/bin/env python

import time
import click
import sys
from tqdm import tqdm

@click.command()
@click.argument('arg1')
@click.argument('arg2')
@click.option('--loops', default=0, help='Loop with progress bar')
@click.option('--exit', default=0, help='Specifiy exit code')
@click.option('--verbose', is_flag=True, help='Print sys info')
@click.option('--query', is_flag=True, help='Prompt for exit code')


def hello(arg1, arg2, exit=0, query=False, verbose=False, loops = 0):
    """Simple script that plays with click."""
    if verbose:
        print(sys.version)
        print(arg1, arg2)
    if loops:
        for j in tqdm(range(loops)):
            time.sleep(1)
    if exit:
        sys.exit(exit)

    if query:
        # demand input from the user
        es = input('specify exit value: ')
        e = int(es)
        sys.exit(e)
        

hello()


# click-zip examples

In [ ]:
week6$ click-zip 
Usage: click-zip [OPTIONS] SEARCHSTRING
Try "click-zip --help" for help.

Error: Missing argument "SEARCHSTRING".

week6$ click-zip --help
Usage: click-zip [OPTIONS] SEARCHSTRING

  Python help script

Options:
  -c, --counts  total number of search hits
  -f, --files   number of hits>0 in each file
  -g, --grep    print lines with search hits
  --help        Show this message and exit.
    
week6$ click-zip -c rpartition
13

week6$ click-zip -f rpartition
whatsnew/2.7.txt : 1
whatsnew/changelog.txt : 2
whatsnew/2.5.txt : 4
whatsnew/3.2.txt : 2
library/importlib.txt : 1
library/stdtypes.txt : 3

week6$ click-zip -g rpartition
whatsnew/2.7.txt(916): * The "split()", "replace()", "rindex()", "rpartition()", and
whatsnew/changelog.txt(3334): * bpo-20047: Bytearray methods partition() and rpartition() now
whatsnew/changelog.txt(7251): * bpo-20047: Bytearray methods partition() and rpartition() now
whatsnew/2.5.txt(975): "rpartition(sep)" methods that simplify a common use case.
whatsnew/2.5.txt(984): "rpartition(sep)" also returns a 3-tuple but starts searching from
whatsnew/2.5.txt(995): >>> 'www.python.org'.rpartition('.')
whatsnew/2.5.txt(997): >>> 'www.python.org'.rpartition(':')
whatsnew/3.2.txt(2379): extension = name.rpartition('.')[2]
whatsnew/3.2.txt(2419): also used by "rfind()", "rindex()", "rsplit()" and "rpartition()".
library/importlib.txt(1673): parent_name, _, child_name = absolute_name.rpartition('.')
library/stdtypes.txt(1549): str.rpartition(sep)
library/stdtypes.txt(2319): bytes.rpartition(sep)
library/stdtypes.txt(2320): bytearray.rpartition(sep)



# Paste your click-zip script in cell below

In [ ]:
# click-zip code:

#!usr/bin/env python

import zipfile
import sys
import click

# Help class for parsing zipfiles using ZipFile
class Help:
    def __init__(self, zippath):
        self.path = zippath[:]
        self.data = {}
        
        # store list of files found in the stream - store in unicode not binary
        # strip out the root dir - also don't save dirs
        with zipfile.ZipFile(self.path) as zstream:
            for file in zstream.infolist():
                if file.is_dir(): continue
                
                # not a dir so now append to self.data but strip ziproot
                fname = file.filename.split('/')
                fname = '/'.join(fname[1:])
                
                self.data[fname] = []
                
                with zstream.open(file) as fdata:
                    for line in fdata.readlines():
                        self.data[fname].append(line.decode('utf-8'))
    
    def counts(self, substring):
        count = 0
        for fname, data in self.data.items():
            for line in data:
                if substring in line:
                    count += 1
        print(count)
        return None
    
    def files(self, substring):
        fnames = {}
        
        for fname, data in self.data.items():
            count = 0
            for line in data:
                if substring in line:
                    count += 1
                    fnames.update({fname: count})
        
        for fname, count in fnames.items():
            print(f'{fname} : {count}')
        return None
    
    def grep(self, substring):
        for fname, data in self.data.items():
            linenum = -1
            for line in data:
                linenum += 1
                if substring in line:
                    print(f'{fname}({linenum}): {line.strip()}')
        return None

# click-zip args
@click.command()
@click.argument('SEARCHSTRING')
@click.option('-c','--counts', is_flag=True, help='total number of search hits')
@click.option('-f','--files', is_flag=True, help='numbers of hits>0 in each file')
@click.option('-g','--grep', is_flag=True, help='print lines with search hits')
#@click.option('-h','--help', help='Show this message and exit')

# click-zip main
def clickZip(searchstring,counts=False,files=False,grep=False):
    zippath = 'python-3.7.2-docs-text.zip'
    h = Help(zippath)

    if counts:
        h.counts(searchstring)
    if files:
        h.files(searchstring)
    if grep:
        h.grep(searchstring)

# run click-zip
clickZip()



# Problems 3 & 4 - define new methods on polylist and polydict
- in class, we discussed two different ways to represent a polynomial
    - polylist, a 'dense' represenation, that hold the coefficients in a list
    - polydict, a 'sparse' representation, that holds (exponent, coefficent) pairs in a dict
- add a method, 'topolydict()' to class 'polylist', that converts the polylist into a polydict
- add a method, 'topolylist()' to class 'polydict', that converts the polydict into a 
polylist
    - note that topolydict will always work, but topolylist can fail, because a polylist cannot represent negative exponents. in this case, raise a ValueError
- add a method, ```__mul__``` to class 'polydict', which implements the '*' operator
    - see examples below

- just to tell them apart, polylist prints with a leading '+'

In [165]:
import functools

class polylist: 
    ''' 
    list poly representation
    coe[n] is the coefficient of the X**n term
        
    [1,2,3] <=> + 3*X**2 + 2*X + 1
    '''
    def __init__(self, coe):
        self.coe = coe

    def termString(self, c , e):
        cs = str(c)
        if c > 0:
            cs = '+ ' + cs
        if (e == 0):
            return(cs)
        if (e == 1):
            return('%s*X' % cs)    
        return('%s*X**%d' % (cs,e))
        
    def __str__(self):
        # print math style
        if self.coe == []:
            return('0')
        terms = [self.termString(c,e) 
            for e,c in enumerate(self.coe) 
            if c != 0]
        terms.reverse()
        s = (' '.join(terms))
        # get rid of leading + 
        return(s)
        
    def __repr__(self):
        return(self.__str__())

    def __len__(self):
        # number of non zero terms
        # 0 len => bool false
        return(len(self.coe) - self.coe.count(0))

    def __add__(self, p2):
        p1len = len(self.coe)
        p2len = len(p2.coe)
        pad = p2len - p1len
        c1 = self.coe
        c2 = p2.coe
        
        if pad < 0:
            c1, c2 = c2, c1
            pad = -pad
    
        c1 = c1[:]
        
        c1.extend([0]*pad)
    
        return(polylist([t1+t2 for t1,t2 in zip(c1,c2)]))
    
    # don't allow a hash
    __hash__ = None    
    
    def evaluate(self, n):
        sum = 0
        for e,c in enumerate(self.coe):
            sum += c*n**e
        return(sum)

    def differentiate(self):
        return(polylist([e*c for e,c in enumerate(self.coe)][1:]))
    
    def integrate(self):
        pi = [c/(e+1.) for e,c in enumerate(self.coe)]
        pi.insert(0,0)
        return(polylist(pi))
    
    def __mul__(self, p2):
        sums = []
        for e1,c1 in enumerate(self.coe):
            prod = [c1 * c2 for c2 in p2.coe]
            for rpt in range(e1):
                prod.insert(0, 0)
            sums.append(polylist(prod))
        return(functools.reduce(polylist.__add__, sums))
    
    def topolydict(self):
        # copy self.coe list
        plist = self.coe[:]
        pdict = {}
        
        # now convert - go through list and add to dict
        for i, n in enumerate(plist):
            pdict.update({i:n})
            
        return polydict(pdict)
            
   

In [440]:
class polydict:
    '''sparse poly representation using a dict
        sparse is {exponent:coefficient, ...}
        only non-zero terms appear in the dict
        
        {2:3, 1:2, 0:1} <=> 3*X**2 + 2*X + 1
    '''
    def __init__(self, d={}):

        # why the copy??
        self.sparse = d.copy()

    def printTerm(self, c ,e):
        cs = str(c)
        if c > 0:
            cs = '+ ' + cs
        if (e == 0):
            return(cs)
        if (e == 1):
            return('%s*X' % cs)    
        return('%s*X**%d' % (cs,e))   
        
    def __str__(self):
        if len(self.sparse) == 0:
            return('0')
        terms = [self.printTerm(self.sparse[e],e) 
                for e in sorted(self.sparse.keys(), 
                                reverse=True) 
                    if self.sparse[e] != 0]
        s = ' '.join(terms)
        if '+ ' == s[0:2]:
            s = s[2:]
        return (s)
    
    def __repr__(self):
        return(self.__str__())

    # don't allow a hash
    __hash__ = None  
    
    def __len__(self):
        return(len(self.sparse))

    # can explicity define bool
    def __bool__(self):
        return(False if len(self.sparse)==0 else True)
        
    def __iter__(self):
        # return a generator function that will
        # iterate thru (exp, coe) pairs
        return( (i for i in self.sparse.items() ))

    # should check types
    def __eq__(self, other):
        return(self.sparse == other.sparse)
        
    def __ne__(self, other):
        return(self.sparse != other.sparse)
        
    # define comparsion to be value of poly at 1
    def __lt__(self, other):
        return(self.evaluate(1) < other.evaluate(1))
        
    def __le__(self, other):
        return(self.evaluate(1) <= other.evaluate(1))
        
    # does poly 'contain' an exponent?
    def __contains__(self, e):
        return(e in self.sparse)
                
    def evaluate(self, n):
        '''eval poly at x=n'''
        sum = 0
        for e in self.sparse.keys():
            sum += self.sparse[e]*n**e
        return(sum)
            
    def __add__(self, p2):
        '''add two polys'''
        n = self.sparse.copy()
        for k,v in p2.sparse.items():
            if None == n.get(k):
                n[k] = v
            else:
                n[k] += v
        return(polydict(n))
        
    def __getitem__(self, index):
        '''pull out terms of the poly
           p[2], p[2:5]
           '''
        keys = sorted(self.sparse.keys(), reverse=True)
        if isinstance(index, int):
            # if asked for a single term, p[n], index will
            # be an int
            inds = [index]
        if isinstance(index, slice):
            # if asked for a slice, p[n:m], index will be
            # a 'slice' object
            inds = range(*index.indices(len(keys)))
        d = {}
        for i in inds:
            e = keys[i]
            d[e] = self.sparse[e]
        return(polydict(d))
        
    def differentiate(self):
        d = {}
        for e,c in self.sparse.items():
            if e != 0:
                d[e-1] = c * e
        return(polydict(d))
    
    def integrate(self):
        d = {}
        for e,c in self.sparse.items():
            d[e+1] = c /(e+1.)
        return(polydict(d))
    
    def topolylist(self):
        plist = []
        
        # 0's are not represented in the dict but I need a elem[0] so let's make one
        if 0 not in self.sparse.keys():
            plist.append(0)
        # I can't count on the dict to be in order so I can append to the list
        for k,v in sorted(self.sparse.items()):
            plist.append(v)
            if k < 0:
                raise ValueError(f'Negative exponent = {k}')
        
        return polylist(plist)
    
    def __mul__(self, d):
        outpoly = {}
        n = self.sparse.copy()
        
        # sort the dicts then for each in each create new values
        if type(d) == polydict:
            if len(d) <= len(n):
                for k,v in sorted(n.items()):
                    for e,c in sorted(d.sparse.items()):
                        newexp = k+e
                        newcoe = v*c
                        # check for like terms, if none update otherwise add
                        if newexp not in outpoly.keys():
                            outpoly.update({newexp:newcoe})
                        else:
                            outpoly[newexp] += newcoe
                return polydict(outpoly)
            # same as above but flipped for list d larger than n
            if len(d) > len(n):
                for k,v in sorted(d.sparse.items()):
                    for e,c in sorted(n.items()):
                        newexp = k+e
                        newcoe = v*c
                        outpoly.update({newexp:newcoe})
                return polydict(outpoly)
        # d is an int so just mutiply
        elif type(d) == int:
            for k,v in n.items():
                n[k] *= d
            return polydict(n)
        else:
            raise TypeError(f'Multiplicand not a polydict or int')

In [441]:
# test polys

pl1 = polylist([1, 2, 3])
pl2 = polylist([0, 10, 5])
pd1 = polydict({2:3, 1:2, 0:1})
pd2 = polydict({1:10, 2:5})
pd3 = polydict({-1:10, 2:5})

pl1, pl2, pd1, pd2, pd3

(+ 3*X**2 + 2*X + 1,
 + 5*X**2 + 10*X,
 3*X**2 + 2*X + 1,
 5*X**2 + 10*X,
 5*X**2 + 10*X**-1)

In [442]:
# conversion examples 

pl1.topolydict(), pl2.topolydict(), pd1.topolylist(), pd2.topolylist()

(3*X**2 + 2*X + 1, 5*X**2 + 10*X, + 3*X**2 + 2*X + 1, + 5*X**2 + 10*X)

In [443]:
# multiply polydict by polydict examples

pd1, pd2, pd3, pd1 * pd2, pd1 * pd3, pd2 * pd3

(3*X**2 + 2*X + 1,
 5*X**2 + 10*X,
 5*X**2 + 10*X**-1,
 15*X**4 + 40*X**3 + 25*X**2 + 10*X,
 15*X**4 + 10*X**3 + 5*X**2 + 30*X + 20 + 10*X**-1,
 25*X**4 + 50*X**3 + 50*X + 100)

In [444]:
# multiply polydict by a number examples

pd1 * 4, pd2 * 3

(12*X**2 + 8*X + 4, 15*X**2 + 30*X)

In [425]:
# can't convert negative exponents in a polydict to polylist

try:
    pd3.topolylist()
except Exception as e:
    print(type(e), e)

<class 'ValueError'> Negative exponent = -1


In [426]:
# can't multiply by a string 

try:
    pd3 * 'asdf'
except Exception as e:
    print(type(e), e)

<class 'TypeError'> Multiplicand not a polydict or int


# Problem 5
- define a function decorator '@secure'
- @secure adds two required arguments before any others, a 'user' and a 'password'
- if the user is not registered, raise LoginFailed error
- if the user is registered, but the password is wrong, raise LoginFailed
- if user/pw ok, then call the function defined below @secure
with its usual arguments

In [427]:
class LoginFailure(Exception):
    pass

In [428]:
# the user/password 'database' that @secure will refer to

up = {}
up['jack'] = 'jackpw'
up['jill'] = 'jillpw'

In [429]:
class secure:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, user, password, *args, **kwargs):
        if user not in up:
            raise LoginFailure(f'User {user} not registered')
        elif password != up[user]:
            raise LoginFailure(f'Bad password for {user}')
            
        val = self.func(*args, **kwargs)
        return val
        

In [430]:
# decorate foo and bar with @secure

@secure
def foo(a,b):
    return (a,b)

@secure
def bar(a=1, b=34):
    return (a, b)

In [431]:
# wrong number of args


try:
    foo()
except TypeError as e:
    print(e)


__call__() missing 2 required positional arguments: 'user' and 'password'


In [432]:
# wrong number of args
# user =1, pw=2

try:
    foo(1,2)
except LoginFailure as e:
    print(e)


User 1 not registered


In [433]:
# good call - user/pw, then regular foo args

foo('jack', 'jackpw', 1 ,2)

(1, 2)

In [434]:
# bad user

try:
    foo('frank', 'bad', 1 ,2)
except LoginFailure as e:
    print( e)


User frank not registered


In [435]:
# good user, bad passwd

try:
    foo('jill', 'guess')
except LoginFailure as e:
    print(e)

Bad password for jill


In [436]:
# keyword args work correctly

bar('jill', 'jillpw')


(1, 34)

In [437]:
bar('jill', 'jillpw', 5, 4)

(5, 4)

In [438]:
bar('jill', 'jillpw', b=12)

(1, 12)

In [439]:
bar('jill', 'jillpw', b=12, a=22)

(22, 12)

# Just for fun...
- look at the ```__mul__``` method on polylist(completely different from the one you have to write for polydict)
- how does it work?
- what does 'reduce' do?

In [ ]:
    def __mul__(self, p2):
        sums = []
        for e1,c1 in enumerate(self.coe):
            prod = [c1 * c2 for c2 in p2.coe]
            for rpt in range(e1):
                prod.insert(0, 0)
            sums.append(polylist(prod))
        return(functools.reduce(polylist.__add__, sums))
   